# This notebook aims to rename folders to proper names and formats.
# This notebook contains the following sections:

## Rename Directories and files altogether

### Run all the cells one time then put the preferred values in the last cell and run it. It will call all the functions in the previous cells

-Importing modules

In [1]:
import os
import tables as tb
import glob
from copy import deepcopy
from datetime import datetime,timedelta
import numpy as np
import xmltodict

def find_file(path, extension=['.raw.kwd']):
    """
    This function finds all the file types specified by 'extension' (ex: *.dat) in the 'path' directory
    and all its subdirectories and their sub-subdirectories etc., 
    and returns a list of all dat file paths
    'extension' is a list of desired file extensions: ['.dat','.prm']
    """
    if type(extension) is str:
        extension=extension.split()   #turning extension into a list with a single element
    return [os.path.join(walking[0],goodfile) for walking in list(os.walk(path)) 
         for goodfile in walking[2] for ext in extension if goodfile.endswith(ext)]

## Renaming

-Functions and classes required for renaming all the OpenEphys files and folders

In [2]:
def rename_folder(folderToRemove,newFolder):
    '''
    folderToRemove (=old), newFolder: full paths to folders
    merge the folders if "newFolder" already exists
    files are renamed (example: folderToRemove_1.txt becomes newFolder_1.txt)
    '''
    #if the new folder doesn't exist, create it
    if not os.path.exists(newFolder):
        os.mkdir(newFolder) 
    #move and rename the files 
    for f in os.listdir(folderToRemove):
        oldpath=os.path.join(folderToRemove,f)
        oldFolderName=os.path.basename(folderToRemove.rstrip(os.sep))
        newFolderName=os.path.basename(newFolder.rstrip(os.sep))
        f=f.replace(oldFolderName,newFolderName)
        newpath=os.path.join(newFolder,f)
        if os.path.exists(newpath):
            os.remove(oldpath)
        else:
            os.rename(oldpath,newpath)
    os.rmdir(folderToRemove)
    
def get_folders_matching_format(rootFolder,globFormat):
    '''
    rootFolder: full path where to look for session folders (ex: "/data/Rat034/Experiments")
    '''
    fullFormat=os.path.join(rootFolder,globFormat)
    return [os.path.basename(f) for f in glob.glob(fullFormat)]

def get_regular_name(wrongName,wrongTimeFormat,regularTimeFormat):
    '''
    from a session name in wrong format, return the regular format
    '''
    date=datetime.strptime(wrongName,wrongTimeFormat)
    return date.strftime(regularTimeFormat)

def get_date(name,timeFormat):
    '''
    return a datetime object from a session name and a format
    '''
    date=datetime.strptime(name,timeFormat)
    return date

def get_regular_name_from_date(date,regularTimeFormat):
    '''
    return a name given a date and a format
    '''
    return date.strftime(regularTimeFormat)

def rename_all_in_rootFolder(rootFolder,wrongGlobFormat,wrongTimeFormat,regularGlobFormat,
                             regularTimeFormat,minuteDelay=2,verbose=False):
    '''
    rootFolder: full path where to look for session folders (ex: "/data/Rat034/Experiments")
    wrongGlobFormat: glob pattern to search for the wrong session names
    wrongTimeFormat: datetime pattern to read date from wrong session names
    regularGlobFormat, regularTimeFormat: glob and datetime patterns for the right session names
    minuteDelay: if two folders have the same date (give or take minuteDelay), merge them
    '''
    delay=timedelta(minutes=minuteDelay)

    #get names and date of the regular folders
    regularFolders=get_folders_matching_format(rootFolder,regularGlobFormat)
    allRegularDates=[get_date(name,regularTimeFormat) for name in regularFolders]
    
    #get names of wrong folders
    fList=get_folders_matching_format(rootFolder,wrongGlobFormat)
    for f in fList:
        merge=False
        #check if there is a regular folder around the same date
        date=get_date(f,wrongTimeFormat)
        for otherDate in allRegularDates:
            if abs(date-otherDate)<delay:
                date=otherDate
                merge=True
                break
        #new name
        newFolder=get_regular_name_from_date(date,regularTimeFormat)
        #rename/merge
        newPath=os.path.join(rootFolder,newFolder)
        oldPath=os.path.join(rootFolder,f)
        rename_folder(oldPath,newPath)
        if verbose:
            if merge:
                print("Merged %s into %s"%(f,newFolder))
            else:
                print("Renamed %s in %s"%(f,newFolder))
                
def rename_files_to_match_folder(folderPath,extensionList):
    '''
    folderPath= "/data/Rat034/Experiments/Rat034_2015_etc"
    extensionList= [".dat", ".prm", ".kwik"]
    --> Renames "someFile.dat" into "Rat024_2015_etc.dat"
    '''
    folderName=os.path.basename(folderPath.rstrip(os.sep))
    extensionList=[ext if ext.startswith(".") else "."+ext for ext in extensionList]
    for f in os.listdir(folderPath):
        for ext in extensionList:
            if f.endswith(ext):
                oldPath=os.path.join(folderPath,f)
                newName=folderName+ext
                newPath=os.path.join(folderPath,newName)
                os.rename(oldPath,newPath)
                break


-Function to rename the files and folders of OpenEphys as a Batch

In [3]:
def rename_batch(root,animalList,verbose,minuteDelay,extensionList):
    
    WRONG_FORMATS={
    "openEphys":(
        "animal_20{0}-{0}-{0}_{0}-{0}-{0}".format("[00-99]?"),    #ex: 'Rat001_2022-02-21_11-28-17'
        "animal_%Y-%m-%d_%H-%M-%S"
    ),
    "seconds":(
        "animal_20{0}_{0}_{0}_{0}_{0}_{0}".format("[00-99]?"),    #ex: 'Rat001_2013_12_26_20_47_54'
        "animal_%Y_%m_%d_%H_%M_%S"
    ),
    "tiret":(
        "animal_20{0}_{0}_{0}-{0}_{0}".format("[00-99]?"),        #ex: 'Rat001_2039_10_15-06_27'
        "animal_%Y_%m_%d-%H_%M"
    ),
    "tiretSeconds":(
        "animal_20{0}_{0}_{0}-{0}_{0}_{0}".format("[00-99]?"),    #ex: 'Rat001_2033_06_23-21_24_46'
        "animal_%Y_%m_%d-%H_%M_%S"
    ),
    "openEphys2":(
        "20{0}-{0}-{0}_{0}-{0}-{0}".format("[00-99]?"),           #ex: '2015-02-15_11-59-48'
        "%Y-%m-%d_%H-%M-%S"
    ),
    }
    # The right format 
    REGULAR_FORMAT=("animal_20{0}_{0}_{0}_{0}_{0}".format("?[00-99]"),  #ex: 'Rat001_2039_10_15_06_27'
                "animal_%Y_%m_%d_%H_%M")
      
    for animal in animalList:
        rawFileList=find_file(os.path.join(root,animal),extensionList)
        rootFolderList=[os.path.split(os.path.split(rawFile)[0])[0] for rawFile in rawFileList]
        #rootFolderList keeps paths to the directories within 'root' which contain session folders
        rootFolderList=list(set(rootFolderList))#to keep unique members
        
        for rootFolder in rootFolderList:
            print("animal %s: %s"%(animal,rootFolder))
    
            #RENAME
            regularGlob=REGULAR_FORMAT[0].replace("animal",animal)
            regularTime=REGULAR_FORMAT[1].replace("animal",animal)
            for name in WRONG_FORMATS:
                globFormat=WRONG_FORMATS[name][0].replace("animal",animal)
                timeFormat=WRONG_FORMATS[name][1].replace("animal",animal)       
                #print(get_folders_matching_format(rootFolder,globFormat))
                rename_all_in_rootFolder(rootFolder,globFormat,timeFormat,regularGlob,regularTime,
                                         minuteDelay=minuteDelay,verbose=verbose)
    
            #CONVERT
            #for every session folder of the animal
            for folder in os.listdir(rootFolder):
                if not folder.startswith(animal):
                    continue
                print("*"+folder)
                path=os.path.join(rootFolder,folder)
                #rename files
                rename_files_to_match_folder(path,extensionList)

                for f in os.listdir(path):
                    fpath=os.path.join(path,f)
                    #convert raw.kwd
                    if f=="settings.xml":
                        newPath=os.path.join(path,"settingsOpenEphys.xml")
                        os.rename(fpath,newPath)
    
            print("\nRenaming done")
            print("--------")


#-----------------------------------------------------------------------------------------------
if "__file__" not in dir():
    root="/data/SWI002/22/"
    #Animal where to run the script
    animalList=["SWI002"]

    #Whether to print "X renamed in Y"
    verbose=True  

    #If a wrong folder and a regular folder are less than "minuteDelay" apart, they are merged
    # example: if minuteDelay=2, 'Rat001_2034_04_22-04_26_12' would be merged with 'Rat001_2034_04_22_04_27'
    minuteDelay=0

    #File to rename (ex: "someFile.dat" -> "Rat024_2015_etc.dat")
    #extensionList=[".dat","+6.raw.kwd",".nrs",".kwx",".kwik",".prm",".prb"]
    extensionList=[".dat",".raw.kwd",".nrs",".kwx",".kwe",".eeg"]
    
    #--------------------------------------------------------------------------------
    rename_batch(root,animalList,verbose,minuteDelay,extensionList)

# ==========================================================
# Script to Rename
##### 'root' must be a directory containing seperate folders for each animal(ex: /data/ containing /data/Rat001, /data/Rat002, ...)
##### 'animalList' determines on which folders within the 'root' this notebook will operate(ex: Rat001, Rat002, ...).
##### renaming MUST precede converting/saving

In [4]:
if "__file__" not in dir():
    #The directory containing separate folders for each animal
    root="E:\\Data\\"
    #in Windows paths must be like this: root="C:\\Data\\Recordings\\" (double backslash instead of single)
    
    #Animal where to run the script
    animalList=["Rat105"]
    
    #Whether to print "X renamed in Y"
    verbose=True  

    #If a wrong folder and a regular folder are less than "minuteDelay" apart, they are merged
    # example: if minuteDelay=2, 'Rat001_2034_04_22-04_26_12' would be merged with 'Rat001_2034_04_22_04_27'
    minuteDelay=0

    #Files to rename (ex: "someFile.dat" -> "Rat024_2015_etc.dat")
    #extensionList=[".dat","+6.raw.kwd",".nrs",".kwx",".kwik",".prm",".prb"]
    extensionList=[".dat",".raw.kwd",".nrs",".kwe",".eeg"]
    
    #--------------------------------------------------------------------------------
    rename_batch(root,animalList,verbose,minuteDelay,extensionList)

animal Rat105: E:\Data\Rat105\Experiments
Renamed 2016-12-07_15-09-42 in Rat105_2016_12_07_15_09
Renamed 2016-12-07_17-04-17 in Rat105_2016_12_07_17_04
Renamed 2016-12-07_17-37-21 in Rat105_2016_12_07_17_37
*Rat105_2016_12_05_17_57
*Rat105_2016_12_06_11_13
*Rat105_2016_12_07_15_09
*Rat105_2016_12_07_17_04
*Rat105_2016_12_07_17_37

Renaming done
--------
